In [2]:
# Import delle librerie richieste
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from matplotlib import pyplot as plt
import numpy as np
from torchvision import datasets, transforms
import torchvision

In [3]:
# Trasformazioni per il training e il testing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        # transforms.Resize(256),
        # transforms.CenterCrop(224), 
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        # transforms.Resize(256),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

Caricamento del Dataset

In [4]:
# Percorsi delle cartelle del dataset
data_dir = 'data\dataset\Armocromia\ARMOCROMIA 1. TEST_TRAIN'
batch_size = 64

# Creazione degli ImageFolder per il training e il testing
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'TRAIN'), data_transforms['train']),
    'test': datasets.ImageFolder(os.path.join(data_dir, 'TEST'), data_transforms['test']),
}

# Creazione dei DataLoader
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False, num_workers=4),
}

# Numero di classi (armocromia)
num_classes = len(image_datasets['train'].classes)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dswal\AppData\Local\Temp\ipykernel_16704\1658227235.py:2: SyntaxWarning: invalid escape sequence '\d'
  data_dir = 'data\dataset\Armocromia\ARMOCROMIA 1. TEST_TRAIN'


In [50]:
# Verifica del caricamento dei dati
def imshow(inp, title=None):
    """Visualizza un tensor di immagini"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # Pausa per aggiornare la trama

def verify_dataloaders(dataloaders):
    for phase in ['train', 'test']:
        print(f"Verifica {phase} set:")
        dataset = dataloaders[phase].dataset
        print(f"Numero di immagini: {len(dataset)}")
        print(f"Numero di classi: {len(dataset.classes)}")
        print(f"Classi: {dataset.classes}")

        # Controlla alcune immagini
        data_iter = iter(dataloaders[phase])
        images, labels = next(data_iter)
        print(f"Dimensioni batch: {images.size()}")  # Dovrebbe essere (32, 3, 224, 224)
        print(f"Etichette batch: {labels}")

        # Visualizza alcune immagini
        out = torchvision.utils.make_grid(images[:4])
        imshow(out, title=[dataset.classes[x] for x in labels[:4]])

verify_dataloaders(dataloaders)

Verifica train set:
Numero di immagini: 1835
Numero di classi: 12
Classi: ['autunno deep', 'autunno soft', 'autunno warm', 'inverno bright', 'inverno cool', 'inverno deep', 'primavera bright', 'primavera light', 'primavera warm', 'summer cool', 'summer light', 'summer soft']


KeyboardInterrupt: 

Definizione del modello ResNET50 e modifica dell'ultimo stato

In [9]:
# Caricamento del modello ResNet-50 pre-addestrato
model = models.resnet50(pretrained=True)

# Congelamento dei parametri iniziali
for param in model.parameters():
    param.requires_grad = False

# Modifica dell'ultimo layer per adattarlo al numero di classi dell'armocromia
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Spostamento del modello sulla GPU se disponibile
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda:0


Definizione della loss function e dell'ottimizzatore

In [10]:
# Definizione della loss function (CrossEntropy) e dell'ottimizzatore (Adam)
learning_rate = 0.0001
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = learning_rate)

Addestramento del modello

In [11]:
# Specifica il percorso della cartella
save_dir = 'model_fine_tuning\\3rd'
os.makedirs(save_dir, exist_ok=True)

In [12]:
# Specifica il percorso del file di log
log_path = os.path.join(save_dir, 'training_log.txt')

# Apri il file di log in modalità append
with open(log_path, 'a') as log_file:
    # Scrivi le informazioni di inizio allenamento
    log_file.write(f'\n\nStarting training\nBatch Size: {batch_size}\nNumber of Epochs: {num_epochs}\nLearning Rate: {learning_rate}\n')
    log_file.write('=' * 20 + '\n')

    for epoch in range(num_epochs):
        log_file.write(f'Epoch {epoch + 1}/{num_epochs}\n')
        log_file.write('-' * 10 + '\n')
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Fase di training e validation
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterazione sui batch di dati
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Azzera i gradienti solo se siamo nella fase di training
                optimizer.zero_grad()

                # Forward pass solo durante il training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass e ottimizzazione solo durante il training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Calcola le statistiche di loss e accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Calcola la loss media e l'accuracy media per epoch
            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            log_file.write(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}\n')
            print(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

    log_file.write('\nTraining complete\n')

Epoch 1/10
----------
train Loss: 2.4754, Acc: 0.0997
test Loss: 2.4435, Acc: 0.1156
Epoch 2/10
----------
train Loss: 2.3982, Acc: 0.1613
test Loss: 2.3952, Acc: 0.1455
Epoch 3/10
----------
train Loss: 2.3369, Acc: 0.2153
test Loss: 2.3538, Acc: 0.1961
Epoch 4/10
----------
train Loss: 2.2896, Acc: 0.2654
test Loss: 2.3178, Acc: 0.2065
Epoch 5/10
----------
train Loss: 2.2389, Acc: 0.3019
test Loss: 2.2851, Acc: 0.2325
Epoch 6/10
----------
train Loss: 2.1966, Acc: 0.3144
test Loss: 2.2532, Acc: 0.2312
Epoch 7/10
----------
train Loss: 2.1550, Acc: 0.3362
test Loss: 2.2264, Acc: 0.2416
Epoch 8/10
----------
train Loss: 2.1207, Acc: 0.3313
test Loss: 2.2004, Acc: 0.2623
Epoch 9/10
----------
train Loss: 2.0862, Acc: 0.3673


KeyboardInterrupt: 

Esplorimo il modello di interesse

In [ ]:
import torch

# Percorso del file .pth
model_path = 'armocromia_resnet50.pth'

# Carica i dati dal file
checkpoint = torch.load(model_path)

# Esplora le chiavi nel checkpoint
print("Chiavi nel checkpoint:")
for key in checkpoint.keys():
    print(key)

# Se il checkpoint contiene i pesi del modello, esplora gli stati
if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint

# Esplora le chiavi nei pesi del modello
print("\nChiavi nel state_dict:")
for key in state_dict.keys():
    print(key)

# Visualizza una parte dei pesi, ad esempio il primo layer
first_layer_weights = state_dict[list(state_dict.keys())[0]]
print("\nPesi del primo layer:")
print(first_layer_weights)

FileNotFoundError: [Errno 2] No such file or directory: 'armocromia_resnet50.pth'